# 2.0-damage-parameter

## Purpose

Estimate the damage and effective damage parameters $w$ and $\overline{w}$.

## Notation

- $w$: Damage parameter
- $\overline{w}$: Effective damage parameter
- $\varepsilon:$ Strain
- $\Delta \varepsilon_t = \varepsilon_{\max} - \varepsilon_{\min}$: total strain range
- $w_{SWT}$: SWT parameter. (Smith–Watson–Topper).
- $\delta w_0$: material parameter in SWT model

## Methodology

The $m$ value is taken from the reference: [29](https://www.sciencedirect.com/science/article/abs/pii/S0013794418302169) of the main paper.

### Met_1:

Attached to equation (4) in the official paper,
$$w = \Delta \varepsilon_t \sigma^{\max}$$
and then computing just one value of $w$ for each CHP specimen (scale: 40, 60, 80, 100), since the files *scale_SS.rpt* contains a global minimum and maximum value.

In this case, since $w_i$ = $w$ for all $i$ then $\overline{w} = w$, since:

$$
\overline{w} = \bigg(\dfrac{1}{V} \sum_{i=1}^{d} \int_{V_i} w_i^m dV_i\bigg)^{1/m} = \bigg(\dfrac{w^m}{V} \sum_{i=1}^{d} \int_{V_i} dV_i\bigg)^{1/m} = \bigg(\dfrac{w^m}{V} \sum_{i=1}^{d} V_i \bigg)^{1/m} =w
$$

**Remark**: *By this way the computation not depend on the parameter $m$.*

### Met_2:

Same as in [Met 1](###met_1) but assuming $R_e = \dfrac{\varepsilon_{min}}{\varepsilon_{\max}} = 0.1 $, and then be able to compute:

$$ \Delta \varepsilon_{i, t} = \varepsilon_{\max, i} - \varepsilon_{\min, i} = 0.9 \varepsilon_{\max, i} $$

and therefore:

$$ w_i = \Delta \varepsilon_{i, t} \cdot \sigma^{\max}$$

In this case, since $w_i$ = $w$ for all $i$ then $\overline{w} = w$, since:

$$
\overline{w} = \bigg(\dfrac{1}{V} \sum_{i=1}^{d} \int_{V_i} w_i^m dV_i\bigg)^{1/m} \approx \bigg(\dfrac{1}{V} \sum_{i=1}^{d} w_i^m V_i \bigg)^{1/m}
$$


### Met_3:

Same as in [Met 2](###met_1) but taking $\varepsilon_{\min, i} = \min(\varepsilon_{\max})$

$$ \Delta \varepsilon_{i, t} = \varepsilon_{\max, i} - \min(\varepsilon_{\max}) $$


### Met 4

Same as in [Met 1](###met_1) but using the equations (1) and (2) on paper [29](https://www.sciencedirect.com/science/article/abs/pii/S0013794418302169), i.e.,
$$w = w_{SWT} - \delta w_0  \quad \text{where} \quad w_{SWT} = \Delta \varepsilon_t \sigma^{\max}$$

### Met 5

Same as in [Met 2](###met_1) but using the equations (1) and (2) on paper [29](https://www.sciencedirect.com/science/article/abs/pii/S0013794418302169), i.e.,
$$w = w_{SWT} - \delta w_0  \quad \text{where} \quad w_{SWT} = \Delta \varepsilon_t \sigma^{\max}$$

## Results

The obtained results are quite different between the approaches, I'm not sure of the viability of them.

## Setup

### Library import

In [3]:
# OS interface
import os
import glob


# Data manipulation
import pandas as pd
import numpy as np
import math
import re


## Options for pandas
pd.options.display.max_columns = 80
pd.options.display.max_rows = 50

# Visualizations


### Parameter definition

In [4]:
# Set input and out path.
print("Current working directory: {0}".format(os.getcwd()))

input_path = os.path.abspath(os.path.join(os.getcwd(), '../data/raw/finite_elements/'))
output_path = os.path.abspath(os.path.join(os.getcwd(), '../data/processed/finite_elements/'))

scale_sizes = ['100', '80', '60', '40']

## Taken from the main papper:
material_param_delta0 = -1.97

## this value is taken from this paper:
# https://www.sciencedirect.com/science/article/abs/pii/S0013794418302169
m = 9.10


Current working directory: c:\Users\tchavarria\DSTI\Python ML Labs\Fatigue-Analysis\notebooks


In [3]:
#!dir

##  Functions 

read_fwf: https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.read_fwf.html

Could be a future read and processes data module.

In [5]:
def create_finite_element_df(file_path, output_path, skiprows = 19, skipfooter = 9):
    '''
    Load the finite element data in an correct format.
    
    '''
    
    if 'SS' in file_path:
        names = ['label', 'LE_max', 'S_max']
        dtype  = {'label': np.int64, 'LE_max': np.float64, 'S_max': np.float64}
    else:
        names = ['label', 'E_vol']
        dtype  = {'label': np.int64, 'E_vol': np.float64}
        
    
    finite_element_df = pd.read_fwf(file_path
                                    , delimiter = '    '
                                    , skiprows = skiprows
                                    , names = names
                                    , index_col = False
                                    , dtype  = dtype
                                    , skipfooter = skipfooter)
    
    ## Make this change in order to fix the label column:
    finite_element_df['label'] = finite_element_df.index + 1
    
    finite_element_df.to_csv(output_path, index = None, sep = ';')
     
   
    
def read_processed_data(file_path):
    '''
    Read file for a given path.
    '''
    processed_df = pd.read_csv(file_path, delimiter = ';')
    return processed_df

## Data processing

In [6]:
# Get FE rpt files paths:
fe_raw_paths = glob.glob(os.path.join(input_path,"**\\*.rpt"),recursive=True)

# Creating processed paths
fe_processed_paths = [path.replace('raw', 'processed').replace('rpt', 'csv') for path in fe_raw_paths]

fe_processed_paths[0]

'c:\\Users\\tchavarria\\DSTI\\Python ML Labs\\Fatigue-Analysis\\data\\processed\\finite_elements\\100_SS.csv'

In [10]:
fe_vol_paths = [vol_path for vol_path in fe_processed_paths if '_Vol' in vol_path]
fe_vol_paths

['c:\\Users\\tchavarria\\DSTI\\Python ML Labs\\Fatigue-Analysis\\data\\processed\\finite_elements\\100_Vol.csv',
 'c:\\Users\\tchavarria\\DSTI\\Python ML Labs\\Fatigue-Analysis\\data\\processed\\finite_elements\\40_Vol.csv',
 'c:\\Users\\tchavarria\\DSTI\\Python ML Labs\\Fatigue-Analysis\\data\\processed\\finite_elements\\60_Vol.csv',
 'c:\\Users\\tchavarria\\DSTI\\Python ML Labs\\Fatigue-Analysis\\data\\processed\\finite_elements\\80_Vol.csv']

In [5]:
# List to separate volume from strain and stress measures
fe_vol_paths = []
fe_ss_paths = []

# Process FE data
for i in range(len(fe_raw_paths)):
    
    fe_processed = fe_processed_paths[i]
    
    if '_Vol' in fe_processed:
        fe_vol_paths.append(fe_processed)
    else:
        fe_ss_paths.append(fe_processed)
        
    print(f'Processing: {fe_processed}')
    create_finite_element_df(fe_raw_paths[i], fe_processed)
    
    


Processing: C:\Users\tchavarria\DSTI\Python ML Labs\Fatigue-Analysis\data\processed\finite_elements\100_SS.csv
Processing: C:\Users\tchavarria\DSTI\Python ML Labs\Fatigue-Analysis\data\processed\finite_elements\100_Vol.csv
Processing: C:\Users\tchavarria\DSTI\Python ML Labs\Fatigue-Analysis\data\processed\finite_elements\40_SS.csv
Processing: C:\Users\tchavarria\DSTI\Python ML Labs\Fatigue-Analysis\data\processed\finite_elements\40_Vol.csv
Processing: C:\Users\tchavarria\DSTI\Python ML Labs\Fatigue-Analysis\data\processed\finite_elements\60_SS.csv
Processing: C:\Users\tchavarria\DSTI\Python ML Labs\Fatigue-Analysis\data\processed\finite_elements\60_Vol.csv
Processing: C:\Users\tchavarria\DSTI\Python ML Labs\Fatigue-Analysis\data\processed\finite_elements\80_SS.csv
Processing: C:\Users\tchavarria\DSTI\Python ML Labs\Fatigue-Analysis\data\processed\finite_elements\80_Vol.csv


### Estimation of the parameters

### Met_1

#### Compute $w$ for each scale size

In [6]:
# Create a data frame with the info of each scale size

w_list1 = []

for ss_path in fe_ss_paths:
    
    # Read data for each scale
    file_name = os.path.basename(ss_path)
    scale_size = re.findall(r'\d+', file_name)[0]
    
    fe_ss_df = read_processed_data(ss_path)
    
    # We need to exponential because the record type is logarithmic strain
    strain_max = math.exp(fe_ss_df['LE_max'].max())
    strain_min = math.exp(fe_ss_df['LE_max'].min())
    
    # Computation of parameters
    sigma_max = fe_ss_df['S_max'].max()
    
    w = (strain_max -strain_min)*sigma_max
    
    scale_info = [scale_size, strain_max, strain_min, sigma_max, w]
    
    w_list1.append(scale_info)
    


w_df1 = pd.DataFrame(w_list1, columns = ['scale_size', 'strain_max', 'strain_min', 'sigma_max', 'w'])

display(w_df1)

,scale_size,strain_max,strain_min,sigma_max,w
0,100,1.002945,1.000083,636.610,1.822035
1,40,1.002784,1.000007,608.429,1.689436
2,60,1.002857,1.000038,623.049,1.756537
3,80,1.002911,1.000089,633.475,1.787181


#### Compute $\overline{w}$ for each scale size

In this case, $\overline{w} = w$.

In [7]:
w_df1['w_effective'] = w_df1['w']
display(w_df1)


,scale_size,strain_max,strain_min,sigma_max,w,w_effective
0,100,1.002945,1.000083,636.610,1.822035,1.822035
1,40,1.002784,1.000007,608.429,1.689436,1.689436
2,60,1.002857,1.000038,623.049,1.756537,1.756537
3,80,1.002911,1.000089,633.475,1.787181,1.787181


### Met_2

#### Compute $w$ for each scale size

In [8]:
# Create a data frame with the info of each scale size

w_dict_met2 = {}

for ss_path, vol_path in zip(fe_ss_paths, fe_vol_paths):
    
    ## Computations of w:
    
    file_name = os.path.basename(ss_path)
    scale_size = re.findall(r'\d+', file_name)[0]
    
    fe_ss_df = read_processed_data(ss_path)
    
    # We need to exponential because the record type is logarithmic strain
    fe_ss_df['strain_max'] = np.exp(fe_ss_df['LE_max'])
    fe_ss_df['strain_min'] = 0.1*fe_ss_df['strain_max']
    
    fe_ss_df['sigma_max'] = fe_ss_df['S_max']
    
    fe_ss_df['w'] = (0.9*fe_ss_df['strain_max'])*fe_ss_df['sigma_max']
    
    fe_ss_df.drop(['LE_max', 'S_max'], axis = 1, inplace = True)
    
    ## Adding the volumne
    
    fe_vol_df = read_processed_data(vol_path)
    
    fe_vol_df.rename(columns = {'E_vol': 'V'}, inplace = True)
    
    ss_vol_df = pd.merge(fe_ss_df, fe_vol_df, how = 'inner', on = 'label', validate = '1:1') 
    
    ## Compute w_i*V_i
    
    ss_vol_df['wV'] = ss_vol_df['w']*ss_vol_df['V']

    w_dict_met2[scale_size] = ss_vol_df
    


### Simple verification

In [9]:
w_dict_met2['100'].shape

(68642, 7)

In [10]:
w_dict_met2['100'].head()

,label,strain_max,strain_min,sigma_max,w,V,wV
0,1,1.000323,0.100032,65.5640,59.026636,1.09385,64.566286
1,2,1.000395,0.100039,81.9723,73.804199,1.09389,80.733675
2,3,1.000445,0.100044,93.0580,83.789467,1.09391,91.658136
3,4,1.000484,0.100048,101.4190,91.321303,1.09393,99.899113
4,5,1.000515,0.100052,108.0430,97.288799,1.09394,106.428108


#### Compute $\overline{w}$ for each scale size

In this case:
$$\overline{w} \approx \bigg(\dfrac{1}{V} \sum_{i=1}^{d} w_i^m V_i \bigg)^{1/m}$$

In [11]:
w_list2 = []

for scale in scale_sizes:
    ## Computation
    
    # w_effective:
    w_effective = (sum(w_dict_met2[scale].wV**m)/sum(w_dict_met2[scale].V))**(1/m)
    
    # average other parameters
    strain_max_avg = np.mean(w_dict_met2[scale].strain_max)
    strain_min_avg = np.mean(w_dict_met2[scale].strain_min)
    sigma_max_avg = np.mean(w_dict_met2[scale].sigma_max)
    w_avg = np.mean(w_dict_met2[scale].w)
    
    scale_info = [scale, strain_max_avg, strain_min_avg
                  , sigma_max_avg
                  , w_avg, w_effective]
    
    w_list2.append(scale_info)
    

w_df2 = pd.DataFrame(w_list2, columns = ['scale_size', 'strain_max_avg'
                                           , 'strain_min_avg', 'sigma_max_avg'
                                           , 'w_avg', 'w_effective'])


display(w_df2)

,scale_size,strain_max_avg,strain_min_avg,sigma_max_avg,w_avg,w_effective
0,100,1.000851,0.100085,178.252852,160.581454,235.185526
1,80,1.000789,0.100079,165.383451,148.980801,225.747727
2,60,1.000669,0.100067,140.114128,126.208311,111.905814
3,40,1.000560,0.100056,117.052945,105.431124,215.628608


### Met_3

#### Compute $w$ for each scale size

In [12]:
# Create a data frame with the info of each scale size

w_dict_met3 = {}

for ss_path, vol_path in zip(fe_ss_paths, fe_vol_paths):
    
    ## Computations of w:
    
    file_name = os.path.basename(ss_path)
    scale_size = re.findall(r'\d+', file_name)[0]
    
    fe_ss_df = read_processed_data(ss_path)
    
    # We need to exponential because the record type is logarithmic strain
    fe_ss_df['strain_max'] = np.exp(fe_ss_df['LE_max'])
    fe_ss_df['strain_min'] = np.min(fe_ss_df['strain_max'])
    
    fe_ss_df['sigma_max'] = fe_ss_df['S_max']
    fe_ss_df['w'] = (fe_ss_df['strain_max'] - fe_ss_df['strain_min'])*fe_ss_df['sigma_max']
    
    
    fe_ss_df.drop(['LE_max', 'S_max'], axis = 1, inplace = True)
    
    ## Adding the volumne
    
    fe_vol_df = read_processed_data(vol_path)
    
    fe_vol_df.rename(columns = {'E_vol': 'V'}, inplace = True)
    
    ss_vol_df = pd.merge(fe_ss_df, fe_vol_df, how = 'inner', on = 'label', validate = '1:1') 
    
    ## Compute w_i*V_i
    
    ss_vol_df['wV'] = ss_vol_df['w']*ss_vol_df['V']

    w_dict_met3[scale_size] = ss_vol_df
    


In [13]:
w_dict_met3['100'].shape

(68642, 7)

In [14]:
w_dict_met3['100'].head()

,label,strain_max,strain_min,sigma_max,w,V,wV
0,1,1.000323,1.000083,65.5640,0.015689,1.09385,0.017161
1,2,1.000395,1.000083,81.9723,0.025535,1.09389,0.027933
2,3,1.000445,1.000083,93.0580,0.033655,1.09391,0.036815
3,4,1.000484,1.000083,101.4190,0.040665,1.09393,0.044484
4,5,1.000515,1.000083,108.0430,0.046663,1.09394,0.051046


#### Compute $\overline{w}$ for each scale size

In this case:
$$\overline{w} \approx \bigg(\dfrac{1}{V} \sum_{i=1}^{d} w_i^m V_i \bigg)^{1/m}$$

In [15]:
w_list3 = []


for scale in scale_sizes:
    ## Computation
    
    # w_effective:
    w_effective = (sum(w_dict_met3[scale].wV**m)/sum(w_dict_met3[scale].V))**(1/m)
    
    # average other parameters
    strain_max_avg = np.mean(w_dict_met3[scale].strain_max)
    strain_min_avg = np.mean(w_dict_met3[scale].strain_min)
    sigma_max_avg = np.mean(w_dict_met3[scale].sigma_max)
    w_avg = np.mean(w_dict_met3[scale].w)
    
    scale_info = [scale, strain_max_avg, strain_min_avg
                  , sigma_max_avg
                  , w_avg, w_effective]
    
    w_list3.append(scale_info)
    

w_df3 = pd.DataFrame(w_list3, columns = ['scale_size', 'strain_max_avg'
                                           , 'strain_min_avg', 'sigma_max_avg'
                                           , 'w_avg', 'w_effective'])


display(w_df3)

,scale_size,strain_max_avg,strain_min_avg,sigma_max_avg,w_avg,w_effective
0,100,1.000851,1.000083,178.252852,0.156134,0.678791
1,80,1.000789,1.000089,165.383451,0.135972,0.650865
2,60,1.000669,1.000038,140.114128,0.112050,0.110606
3,40,1.000560,1.000007,117.052945,0.091932,0.634756


### Met_4

#### Compute $w$ for each scale size

In [16]:
w_df4 = w_df1.copy() 
w_df4['w'] = w_df4['w'] - material_param_delta0
display(w_df4)

,scale_size,strain_max,strain_min,sigma_max,w,w_effective
0,100,1.002945,1.000083,636.610,3.792035,1.822035
1,40,1.002784,1.000007,608.429,3.659436,1.689436
2,60,1.002857,1.000038,623.049,3.726537,1.756537
3,80,1.002911,1.000089,633.475,3.757181,1.787181


#### Compute $\overline{w}$ for each scale size

In this case, $\overline{w} = w$.

In [17]:
w_df4['w_effective'] = w_df4['w']
display(w_df4)


,scale_size,strain_max,strain_min,sigma_max,w,w_effective
0,100,1.002945,1.000083,636.610,3.792035,3.792035
1,40,1.002784,1.000007,608.429,3.659436,3.659436
2,60,1.002857,1.000038,623.049,3.726537,3.726537
3,80,1.002911,1.000089,633.475,3.757181,3.757181


### Met_5

#### Compute $w$ for each scale size

In [18]:
w_dict_met5 = w_dict_met3.copy()

In [19]:
w_list5 = []


for scale in scale_sizes:
    
    ## Adjust w computation:
    
    w_dict_met5[scale]['w'] = w_dict_met5[scale]['w'] - material_param_delta0
    w_dict_met5[scale]['wV'] = w_dict_met5[scale]['w']*w_dict_met5[scale]['V']
    
    ## Computation
    
    # w_effective:
    w_effective = (sum(w_dict_met5[scale].wV**m)/sum(w_dict_met5[scale].V))**(1/m)
    
    # average other parameters
    strain_max_avg = np.mean(w_dict_met5[scale].strain_max)
    strain_min_avg = np.mean(w_dict_met5[scale].strain_min)
    sigma_max_avg = np.mean(w_dict_met5[scale].sigma_max)
    w_avg = np.mean(w_dict_met5[scale].w)
    
    scale_info = [scale, strain_max_avg, strain_min_avg
                  , sigma_max_avg
                  , w_avg, w_effective]
    
    w_list5.append(scale_info)
    

    
    
w_df5 = pd.DataFrame(w_list5, columns = ['scale_size', 'strain_max_avg'
                                           , 'strain_min_avg', 'sigma_max_avg'
                                           , 'w_avg', 'w_effective'])


display(w_df5)

,scale_size,strain_max_avg,strain_min_avg,sigma_max_avg,w_avg,w_effective
0,100,1.000851,1.000083,178.252852,2.126134,2.069938
1,80,1.000789,1.000089,165.383451,2.105972,2.145091
2,60,1.000669,1.000038,140.114128,2.082050,1.802031
3,40,1.000560,1.000007,117.052945,2.061932,2.311446


## Save the results

In [20]:
output_results_path = os.path.abspath(os.path.join(os.getcwd(), '../data/processed/parameters/damage_parameter'))


w_df1.to_csv(os.path.join(output_results_path, 'met1.csv'), index = None)
w_df2.to_csv(os.path.join(output_results_path, 'met2.csv'), index = None)
w_df3.to_csv(os.path.join(output_results_path, 'met3.csv'), index = None)
w_df4.to_csv(os.path.join(output_results_path, 'met4.csv'), index = None)
w_df5.to_csv(os.path.join(output_results_path, 'met5.csv'), index = None)



## Suggested next steps


- Check the justification and reasonability of the results
- Select the computation strategy (if there is one correct).
- Improve documentation and assumptions in each method.